In [110]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
from tqdm import tqdm
warnings.filterwarnings("ignore")
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)

In [111]:
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import LinearSVR

In [138]:
file_train_name='./Benz_data/train.csv'
file_test_name='./Benz_data/test.csv'

In [139]:
Train=pd.read_csv(file_train_name,index_col='ID')
Test=pd.read_csv(file_test_name,index_col='ID')
target=Train['y']
Train.drop(['y'],axis=1,inplace=True)

In [140]:
print("Train shape",Train.shape)
print("Test shape",Test.shape)

Train shape (4209, 376)
Test shape (4209, 376)


In [141]:
print("feature number %d"%(Train.shape[1]))
print('-----------------------------------')
Train.dtypes.value_counts()

feature number 376
-----------------------------------


int64     368
object      8
dtype: int64

In [142]:
feature=[[],[],[],[]]
for c in Train.columns:
    t=Train[c].dtype
    u=len(np.unique(Train[c]))
    if t==object:
        feature[0].append(c)
    elif u==1:
        feature[1].append(c)
    elif u==2:
        feature[2].append(c)
    else:
        feature[3].append(c)
    
print("String:%d\nConstant:%d\n2 kind:%d\nother kind:%d"%(len(feature[0]),len(feature[1]),len(feature[2]),len(feature[3])))
print("String features:",feature[0])
print("Constant features:",feature[1])

String:8
Constant:12
2 kind:356
other kind:0
String features: ['X0', 'X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X8']
Constant features: ['X11', 'X93', 'X107', 'X233', 'X235', 'X268', 'X289', 'X290', 'X293', 'X297', 'X330', 'X347']


In [143]:
Full=Train.append(Test)

In [144]:
new_Full=Full.drop(feature[1],axis=1)

pre=OneHotEncoder(sparse=False)
pre.fit(new_Full[feature[0]])
tmp=pre.transform(new_Full[feature[0]])
pre_Full=pd.concat([new_Full,pd.DataFrame(tmp,index=new_Full.index)],axis=1)
Last_Full=pre_Full.drop(feature[0],axis=1)

Train,Test=Last_Full[:4209],Last_Full[4209:]

In [145]:
params={
        'booster':'gbtree',
        'learning_rate': 0.1, 
        'n_estimators': 55, 
        'max_depth': 3, 
        'min_child_weight': 7, 
        'seed': 0,
        'subsample': 0.7, 
        'colsample_bytree': 0.9, 
        'gamma': 0.001, 
        'reg_alpha': 0.2, 
        'reg_lambda': 2
}

# cv_params={
#     'reg_lambda': [1,2,3]
# }

reg_xgb=xgb.XGBRegressor()
reg_gbdt=GradientBoostingRegressor()
# optimized_GBM=GridSearchCV(estimator=reg_xgb, param_grid=cv_params, scoring='r2', cv=30,verbose=1, n_jobs=-1)
# optimized_GBM.fit(Train,target)
# print(optimized_GBM.best_score_)
# print(optimized_GBM.best_params_)    
# reg_xgb.fit(Train,target)

In [150]:
kfold=KFold(n_splits=100)
tmp_train,tmp_test=[[],[]],[[],[]]
for u,v in kfold.split(Train):
    reg_xgb.fit(Train.iloc[u,:],target.iloc[u])
    tmp_train[0].extend(reg_xgb.predict(Train.iloc[v,:]))
    tmp_test[0].append(reg_xgb.predict(Test))

for u,v in kfold.split(Train):
    reg_gbdt.fit(Train.iloc[u,:],target.iloc[u])
    tmp_train[1].extend(reg_gbdt.predict(Train.iloc[v,:]))
    tmp_test[1].append(reg_gbdt.predict(Test))

[18:34:44] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:34:50] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:34:57] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:03] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:09] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:35:22] WARNI

[18:39:55] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:01] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:08] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:14] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:20] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:27] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:40:33] WARNI

[18:45:05] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[18:45:11] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [151]:
new_train=np.array([tmp_train[0],tmp_train[1]])
new_test=np.array([np.mean(tmp_test[0],axis=0),np.mean(tmp_test[1],axis=0)])

In [152]:
reg_xgb.fit(new_train.T,target)
Ans=reg_xgb.predict(new_test.T)

[18:52:35] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [153]:
csv=pd.DataFrame({'ID':Test.index,'y':Ans})
csv.to_csv('XXX.csv',index=False)